# โจทย์: Automate SQL ได้ไหม?


## SQLite with Python

In [23]:
# เปลี่ยนจาก sqlite3 เป็น sqlalchemy
from sqlalchemy import create_engine, text

# สร้าง engine แทนการใช้ connect โดยตรง
engine = create_engine('sqlite:///mysqlite.db')

# สร้างตาราง products ด้วย engine.execute
with engine.connect() as conn:
    conn.execute(text('''
    CREATE TABLE IF NOT EXISTS products (
        product_id INTEGER PRIMARY KEY,
        name TEXT NOT NULL,
        price REAL NOT NULL,
        category TEXT
    )
    '''))

    # สร้างตาราง orders
    conn.execute(text('''
    CREATE TABLE IF NOT EXISTS orders (
        order_id INTEGER PRIMARY KEY,
        product_id INTEGER,
        quantity INTEGER,
        order_date TEXT
    )
    '''))

In [24]:
with engine.connect() as conn:
    # เพิ่มข้อมูลในตาราง products
    conn.execute(text('''
    INSERT INTO products (name, price, category)
    VALUES ('iPhone 14', 35900, 'Electronics')
    '''))
    
    # เพิ่มข้อมูลในตาราง orders 
    conn.execute(text('''
    INSERT INTO orders (product_id, quantity, order_date)
    VALUES (1, 2, '2023-10-20')
    '''))
    
    conn.commit()

## Faker

In [ ]:
from faker import Faker

# สร้าง Faker object สำหรับภาษาไทย
fake = Faker()

fake_data = {
    "name": fake.random_element(elements=("iPhone", "iPad", "MacBook", "AirPods")),
    "price": fake.random_int(min=100, max=50000),
    "category": fake.random_element(
        elements=("Electronics", "Clothing", "Food", "Books")
    ),
}
fake_data

## Faker + SQLite

In [ ]:
# สร้างข้อมูลปลอมด้วย Faker
fake_product = {
    "name": fake.random_element(elements=("iPhone", "iPad", "MacBook", "AirPods")),
    "price": fake.random_int(min=100, max=50000), 
    "category": fake.random_element(elements=("Electronics", "Clothing", "Food", "Books"))
}
fake_product


In [27]:
# เพิ่มข้อมูลลงใน SQLite
with engine.connect() as conn:
    conn.execute(text('''
    INSERT INTO products (name, price, category)
    VALUES (:name, :price, :category)
    '''), fake_product)
    conn.commit()

## Pandas

In [28]:
import pandas as pd

In [ ]:
# สร้าง Series จากข้อมูลตัวเลข
series1 = pd.Series([1, 2, 3, 4, 5])
series1

In [ ]:
# สร้าง Series จากข้อมูลข้อความ
series2 = pd.Series(['แอปเปิ้ล', 'กล้วย', 'ส้ม'])
print(series2)

In [ ]:
# สร้าง DataFrame จากข้อมูลพื้นฐาน
data = {
    'name': ['สมชาย', 'สมหญิง', 'สมศรี'],
    'age': [25, 30, 35],
    'city': ['กรุงเทพ', 'เชียงใหม่', 'ภูเก็ต']
}
df = pd.DataFrame(data)
df

In [ ]:
# แสดงข้อมูลเบื้องต้นของ DataFrame
df.describe()

In [ ]:
# การเข้าถึงคอลัมน์
df['name']

## Pandas + SQLite

In [ ]:
data = {
    'name': ['สมชาย', 'สมหญิง', 'สมศรี'],
    'age': [25, 30, 35],
    'city': ['กรุงเทพ', 'เชียงใหม่', 'ภูเก็ต']
}
df = pd.DataFrame(data)
df

In [ ]:
df.to_sql('people', engine, if_exists='replace', index=False)

## Pandas + SQLite + Faker

In [ ]:
# สร้างข้อมูลปลอมด้วย Faker
products = []   
for _ in range(100):
    product = {
        "name": fake.random_element(elements=("iPhone", "iPad", "MacBook", "AirPods")),
    "price": fake.random_int(min=100, max=50000), 
        "category": fake.random_element(elements=("Electronics", "Clothing", "Food", "Books"))
    }
    products.append(product)

# สร้าง DataFrame สำหรับสินค้า
df_products = pd.DataFrame(products)
df_products

In [ ]:
df_products.to_sql('products', engine, if_exists='append', index=False)

## Challege: สร้างข้อมูลลูกค้าโดยใช้ Faker และ Pandas และเพิ่มลงใน SQLite ทุก 5 วินาที

In [ ]:
import faker
import pandas as pd
import time

# สร้าง Faker object สำหรับสร้างข้อมูลปลอม
fake = faker.Faker()

# สร้างข้อมูลลูกค้า
while True:
    customers = []
    for _ in range(5):  # สร้าง 5 รายการ
        customer = {
            'customer_id': fake.uuid4(),
            'name': fake.name(),
            'email': fake.email(), 
            'phone': fake.phone_number(),
            'address': fake.address(),
            'segment': fake.random_element(['Retail', 'Wholesale', 'Online', 'In-Store']),
            'join_date': fake.date_between(start_date='-2y', end_date='today'),
            'loyalty_points': fake.random_int(min=0, max=1000)
        }
        customers.append(customer)

    # แสดงข้อมูลลูกค้าในรูปแบบ DataFrame
    customer_df = pd.DataFrame(customers)
    customer_df.to_sql('customers', engine, if_exists='append', index=False)
    print(f"Created {len(customers)} customers at {time.strftime('%H:%M:%S')}")
    time.sleep(5)

In [ ]:
import faker
import pandas as pd
import time

# สร้าง Faker object สำหรับสร้างข้อมูลปลอม
fake = faker.Faker()

def generate_customers(num_customers):
    customers = []
    for _ in range(num_customers):  # สร้าง 5 รายการ
        customer = {
            'customer_id': fake.uuid4(),
            'name': fake.name(),
            'email': fake.email(), 
            'phone': fake.phone_number(),
            'address': fake.address(),
            'segment': fake.random_element(['Retail', 'Wholesale', 'Online', 'In-Store']),
            'join_date': fake.date_between(start_date='-2y', end_date='today'),
            'loyalty_points': fake.random_int(min=0, max=1000)
        }
        customers.append(customer)
    return customers

# สร้างข้อมูลลูกค้า
while True:
    customers = generate_customers(5)
    customer_df = pd.DataFrame(customers)
    customer_df.to_sql('customers', engine, if_exists='append', index=False)
    print(f"Created {len(customers)} customers at {time.strftime('%H:%M:%S')}")
    time.sleep(5)